In [13]:
%reload_ext autoreload
%autoreload 2


import os
import sys
import pysam 
import fileinput


sam = pysam.AlignmentFile('to_te.bam', 'r')
read = next(sam)


def bam2bed(fh):
    for read in fh:
        if read.is_unmapped or \
            read.is_supplementary or \
            read.is_secondary:
            next
        # output
        bed = [
            read.refernce_name,
            read.reference_start - 1,
            read.reference_end,
            '{}:{}'.format(read.query_name, read.query_sequence),
            '255',
            '-' if read.is_reverse else '+'
        ]
        bed = list(map(str, bed)) # to str
        yield bed
        

def main():
    # read data
    s = sys.argv[1] if len(sys.argv) > 1 else '-' # stdin
    sam = pysam.AlignmentFile("-", "r")
    # output
    for b in bam2bed(sam):
        print('\t'.join(b))
    



In [30]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from xopen import xopen
from hiseq.utils.utils import log, update_obj, run_shell_cmd
from hiseq.utils.file import fx_name, check_path



def readfq(fh): # this is a generator function
    """
    source: https://github.com/lh3/readfq/blob/master/readfq.py
    processing fastq file
    """
    last = None # this is a buffer keeping the last unprocessed line
    while True: # mimic closure; is it a bad idea?
        if not last: # the first record or a record following a fastq
            for l in fh: # search for the start of the next record
                if l[0] in '>@': # fasta/q header line
                    last = l[:-1] # save this line
                    break
        if not last: break
        name, seqs, last = last[1:].partition(" ")[0], [], None
        for l in fh: # read the sequence
            if l[0] in '@+>':
                last = l[:-1]
                break
            seqs.append(l[:-1])
        if not last or last[0] != '+': # this is a fasta record
            yield name, ''.join(seqs), None # yield a fasta record
            if not last: break
        else: # this is a fastq record
            seq, leng, seqs = ''.join(seqs), 0, []
            for l in fh: # read the quality
                seqs.append(l[:-1])
                leng += len(l) - 1
                if leng >= len(seq): # have read enough quality
                    last = None
                    yield name, seq, ''.join(seqs); # yield a fastq record
                    break
            if last: # reach EOF before reading enough quality
                yield name, seq, None # yield a fasta record instead
                break


def cal_freq(x, strandness=True):
    """Calculate the frequency of list
    ['length', 'strand']
    return dataframe
    """
    header = ['length', 'count']
    if isinstance(x, list):
        df = pd.DataFrame(x, columns=header).groupby(['length']).sum().reset_index()
    else:
        df = pd.DataFrame(columns=header)
    return df

      
def parse_fx_id(x):
    """Extract read count from read id
    if read id in this format:
    1-123 => 123 # count is 123
    abc   => 1 # count 1
    """
    if isinstance(x, str):
        m = re.match('^(\d+)\-(\d+)$', x)
        out = m.group(2) if isinstance(m, re.Match) else 1
        out = int(out) # to int
    elif isinstance(x, list):
        out = [parse_fx_id(i) for i in x]
    else:
        out = None
    return out


x = '/data/yulab/wangming/work/yu_2021/piwi_lxh/results/hiseq/smRNA/results/piRNA_pipe/smRNAseq_DaGal4_0_2h_endo_Piwi_IP_rep1/07.te/demo.fq.gz'

csv_file = None
n_max = 0
chunk = 1000000
frag_size_list = []
counter = 0
frames = []    
with xopen(x) as r:
    for name, seq, qual in readfq(r):
        counter += 1
        n_count = parse_fx_id(name)
        frag_size_list.append([len(seq), n_count])
        # last record
        if n_max > 0 and counter >= n_max:
            log.info('Stopped at limit: {}'.format(n_max))
            break # stop
        # chunk
        if counter > 0 and counter%chunk == 0:
            a = cal_freq(frag_size_list)
            print('!AAAA1', a)
            frames.append(cal_freq(frag_size_list))
            frag_size_list = [] # empty
            log.info('{} : {}'.format('Processed', counter))
    # last chunk
    if len(frag_size_list) > 0:
        a = cal_freq(frag_size_list)
        # print('!AAAA2', a)
        # print('!BBBB2', frag_size_list)
        frames.append(cal_freq(frag_size_list))
        # frag_size_list = [] # empty
        log.info('{} : {}'.format('Processed', counter))
# overall
df = pd.concat(frames, axis=0).groupby(['length']).sum().reset_index()
df['id'] = fx_name(x)
# save to file
if isinstance(csv_file, str):
    if os.path.exists(os.path.dirname(csv_file)):
        df.to_csv(csv_file, index=False)
        
print('!CCCC1', frag_size_list)
        

[2022-02-23 08:45:02 INFO] Processed : 10
!CCCC1 [[29, 1], [29, 1], [23, 1], [23, 1], [26, 8], [24, 4], [27, 2], [26, 1], [26, 1], [26, 1]]


In [35]:

header = ['length', 'count']
pd.DataFrame(frag_size_list, columns=header).groupby(['length']).sum().reset_index()

,length,count
0,23,2
1,24,4
2,26,11
3,27,2
4,29,2
